# Introduction

Suppose we are observing exponential growth $\dot{y} = \theta y$ but we don't know $\theta$ and wish to estimate it. We could assume $\theta \sim {\cal{N}}(\mu, \sigma^2)$ and use something like Markov Chain Monte Carlo or Hamiltonian Monte Carlo and any observations to infer $\mu$ and $\sigma$. However, we might want to model that the further we go into the future, the less we know about $\theta$. We can write our system as as

$$
\begin{aligned}
\mathrm{d}y & = \theta y\mathrm{d}t \\
\mathrm{d}\theta & = \sigma\mathrm{d}W_t
\end{aligned}
$$

where $W_t$ is Brownian Motion.




$\texttt{/Users/dom/Dropbox/Tidy/NumMethHaskell/DynSys.bib}$

@Kantas2015

# Fokker-Planck

$$
d \mathbf{X}_{t}=\boldsymbol{\mu}\left(\mathbf{X}_{t}, t\right) d t+\boldsymbol{\sigma}\left(\mathbf{X}_{t}, t\right) d \mathbf{W}_{t}
$$

$$
\frac{\partial}{\partial t} p(t, \mathbf{x})+\sum_{k=1}^{n} \frac{\partial}{\partial x_{k}}\left({\mu}_{k}(t, \mathbf{x}) p(t, \mathbf{x})\right)=\frac{1}{2} \sum_{j=1, k=1}^{n} \frac{\partial^{2}}{\partial x_{j} \partial x_{k}}\left[\left(\sigma(t, \mathbf{x}) \sigma^{T}(t, \mathbf{x})\right)_{j k} p(t, \mathbf{x})\right]
$$

For our particular system we have

$$
\frac{\partial}{\partial t} p(t, y, \theta)+\frac{\partial}{\partial y}\left({\mu}_{1}(t, y, \theta) p(t, y, \theta)\right)+\frac{\partial}{\partial \theta}\left({\mu}_{2}(t, y, \theta) p(t, y, \theta)\right)=\frac{1}{2}\left[\sigma_{y}^{2} \frac{\partial^{2}}{\partial y^{2}} p(t, y, \theta)+\sigma_{\theta}^{2} \frac{\partial^{2}}{\partial \theta^{2}} p(t, y, \theta)\right]
$$

And since $\mu_1 = \theta y$, $\mu_2 = 0$ and $\sigma_y = 0$ this further simplifies to

$$
\frac{\partial}{\partial t} p(t, y, \theta)+\frac{\partial}{\partial y}(\theta y p(t, y, \theta))=\sigma_{\theta}^{2} \frac{\partial^{2}}{\partial \theta^{2}} p(t, y, \theta)
$$

We can note two things:

* This is an advection / diffusion equation with two spatial variables ($y$ and $\theta$).
* If $\sigma_\theta = 0$ then this is a transport (advection?) equation.

$$
\frac{\partial}{\partial t} p(t, y, \theta)+\frac{\partial}{\partial y}(\theta y p(t, y, \theta))=0
$$

Thus we need to turn our attention to solving advection / diffusion equations.

# Diffusion

Let's use the example given in the [ark examples](https://computing.llnl.gov/sites/default/files/public/ark_examples.pdf) (@hindmarsh2005sundials) as that way we can compare the results to check our implementation.

We have

$$
u_{t}=k u_{x x} + f
$$

The initial condition is

$$u(0, x)=0$$

We allow no heat to escape at the boundaries $x = 0$ and $x = 1$

$$
\frac{\partial u}{\partial t}(t, 0)=\frac{\partial u}{\partial t}(t, 1)=0
$$

We have a point heat source in the middle

$$
f(t, x)=\left\{\begin{array}{ll}{1} & {\text { if } x=1 / 2} \\ {0} & {\text { otherwise }}\end{array}\right.
$$

We can discretize over this spatial dimension using

$$
u_{x x}=\frac{u_{j+1}-2 u_{j}+u_{j-1}}{\Delta x^{2}}
$$

where

$$
u_{j}(t) \triangleq u\left(t, x_{j}\right), \quad x_{j} \triangleq j \Delta x, \quad 0 \leq j \leq n+1
$$

$$
\dot{u}_i = \sum_0^{n+1} A_{i\,j} u_j + B_i, \quad 0 \leq i \leq n+1
$$

where

$$
\begin{aligned}
A_{0\,j}     = 0, & \quad 0 \leq j \leq n+1, & \text{boundary condition} \\
A_{i\,i-1}   = 1  &                          &                           \\
A_{i\,i}     = 2  &                          &                           \\
A_{i\,i+1}   = 1  &                          &                           \\
A_{{n+1}\,j} = 0, & \quad 0 \leq j \leq n+1, & \text{boundary condition} \\
A_{i\,j}     = 0  & \quad \text{otherwise}   &                           \\
\end{aligned}
$$



Converting this to a system of ODEs is straightforward:

$$
\begin{bmatrix}
\dot{u_0} \\
\dot{u_1} \\
\dot{u_2}
\end{bmatrix}
=
\begin{bmatrix}
0 & 0 & 0 \\
1 & 2 & 1 \\
0 & 0 & 0
\end{bmatrix}
\begin{bmatrix}
u_0 \\
u_1 \\
u_2
\end{bmatrix}
+
\begin{bmatrix}
f_0 \\
f_1 \\
f_2
\end{bmatrix}
$$

where $f_j \triangleq f(t, x_j)$.



$$
\begin{bmatrix}
\dot{u_0} \\
\dot{u_1} \\
\dot{u_2}
\end{bmatrix}
=
\begin{bmatrix}
0 & 0 & 0 \\
1 & 2 & 1 \\
0 & 0 & 0
\end{bmatrix}
\begin{bmatrix}
u_0 \\
u_1 \\
u_2
\end{bmatrix}
+
\begin{bmatrix}
f_0 \\
f_1 \\
f_2
\end{bmatrix}
$$


In [2]:
import           Numeric.LinearAlgebra

In [1]:
import           Numeric.Sundials.ARKode.ODE

: 